# Merge Movie Datasets

In this notebook, we merge four distinct movie datasets to create a comprehensive and enriched dataset:

1. **MovieSummaries** dataset: Serves as the primary selection of films.
2. **TMDB (The Movie Database)** dataset: Provides revenue, budget, and additional metadata. Downloaded from [Kaggle](https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies).
3. **IMDB Non-Commercial Dataset**: Provides ratings, genres, and other film attributes. Obtained from [IMDB Non-Commercial Datasets](https://developer.imdb.com/non-commercial-datasets/).
4. **Wikidata Dataset**: Scraped data providing supplementary information about films.

### Merging Process
- We first merge **MovieSummaries** with **TMDB**.
- Then, we merge the **IMDB** and **Wikidata** datasets.
- Finally, the two merged datasets are combined, retaining the selection of films from **MovieSummaries** as the base.

In [1]:
import pandas as pd
import numpy as np
import json
import unicodedata
from datetime import datetime

from auxiliary_functions_for_merging import *

DATA_PATH = "./../../Data/"

# Merge IMDB and TMDB datasets

In [3]:
# Import and clean IMDB dataset, create key movieTitle_year

title_basics = pd.read_csv(DATA_PATH + "title.basics.onlymovies.tsv", sep='\t').drop(columns=["titleType", "endYear"])
title_basics = title_basics[title_basics["isAdult"] == 0].drop(columns=["isAdult"])
title_basics["startYear"] = title_basics["startYear"].apply(lambda x: pd.NA if pd.isna(x) else int(x))
title_basics["runtimeMinutes"] = title_basics["runtimeMinutes"].apply(lambda x: pd.NA if pd.isna(x) else int(x))
title_basics["genres"] = title_basics["genres"].apply(lambda x: pd.NA if pd.isna(x) else x.lower())

title_ratings = pd.read_csv(DATA_PATH + "title.ratings.onlymovies.tsv", sep='\t')
title_basics = pd.merge(title_basics, title_ratings, left_on="tconst", right_on="tconst", how="left", suffixes=('', '_rating')).copy(deep=True)
title_basics["numVotes"] = title_basics["numVotes"].apply(lambda x: pd.NA if pd.isna(x) else int(x))

title_basics["title_year_key"] = create_key_series(title_basics, ["primaryTitle", "startYear"])
duplicate_keys = title_basics["title_year_key"].value_counts()
duplicate_keys = duplicate_keys[duplicate_keys > 1].index
title_basics["title_year_key"] = title_basics["title_year_key"].apply(lambda x: pd.NA if x in duplicate_keys else x)

In [4]:
# Import and clean TMDB dataset, create key movieTitle_year

cols = ["id", "title", "release_date", "revenue", "runtime", "adult", "budget", "imdb_id", "original_language",
        "overview", "genres", "production_companies", "production_countries", "spoken_languages", "keywords"]

TMDB_movie_dataset = pd.read_csv(DATA_PATH + "TMDB_movie_dataset.csv")[cols]
TMDB_movie_dataset = TMDB_movie_dataset[TMDB_movie_dataset["adult"] == 0].drop(columns=["adult"])
TMDB_movie_dataset["genres"] = TMDB_movie_dataset["genres"].apply(lambda x: pd.NA if pd.isna(x) else x.lower().replace(", ",","))
TMDB_movie_dataset["year"] = TMDB_movie_dataset["release_date"].apply(extract_year)
TMDB_movie_dataset["release_date"] = TMDB_movie_dataset["release_date"].apply(lambda x: x if is_valid_date(x) else pd.NA)
TMDB_movie_dataset["production_companies"] = TMDB_movie_dataset["production_companies"].apply(lambda x: pd.NA if pd.isna(x) else x.replace(", ",","))
TMDB_movie_dataset["spoken_languages"] = TMDB_movie_dataset["spoken_languages"].apply(lambda x: pd.NA if pd.isna(x) else x.replace(", ",","))
TMDB_movie_dataset["production_countries"] = TMDB_movie_dataset["production_countries"].apply(lambda x: pd.NA if pd.isna(x) else x.replace(", ",","))
TMDB_movie_dataset["keywords"] = TMDB_movie_dataset["keywords"].apply(lambda x: pd.NA if pd.isna(x) else x.replace(", ",","))

TMDB_movie_dataset["title_year_key"] = create_key_series(TMDB_movie_dataset, ["title", "year"])
duplicate_keys = TMDB_movie_dataset["title_year_key"].value_counts()
duplicate_keys = duplicate_keys[duplicate_keys > 1].index
TMDB_movie_dataset["title_year_key"] = TMDB_movie_dataset["title_year_key"].apply(lambda x: pd.NA if x in duplicate_keys else x)

In [5]:
# Merge IMDB and TMDB datasets first by imdbID and then using the key movieTitle_year

df1 = pd.merge(title_basics, TMDB_movie_dataset, left_on="tconst", right_on="imdb_id", how="inner", suffixes=('_IMDB', '_TMDB'))
rest_of_title_basics = title_basics[~title_basics["tconst"].isin(df1["tconst"])].dropna(subset=["title_year_key"])
df2 = pd.merge(rest_of_title_basics, TMDB_movie_dataset, left_on="title_year_key", right_on="title_year_key", how="left", suffixes=('_IMDB', '_TMDB'))

movies_IMDB_TMDB = pd.concat([df1, df2]).drop(columns=["title_year_key", "title_year_key_IMDB", "title_year_key_TMDB"])

In [6]:
# Combine columns that provide the same information

movies_IMDB_TMDB["startYear"] = movies_IMDB_TMDB.apply(lambda row: row["year"] if pd.isna(row["startYear"]) else row["startYear"], axis=1)
movies_IMDB_TMDB["runtimeMinutes"] = movies_IMDB_TMDB.apply(lambda row: row["runtime"] if pd.isna(row["runtimeMinutes"]) else row["runtimeMinutes"], axis=1)
movies_IMDB_TMDB["genres"] = movies_IMDB_TMDB.apply(lambda row: merge_comma_sep(row["genres_IMDB"], row["genres_TMDB"]), axis=1)
tresh = 50
movies_IMDB_TMDB["budget"] = movies_IMDB_TMDB["budget"].apply(lambda x: pd.NA if pd.isna(x) or x<=tresh else x)
movies_IMDB_TMDB["revenue"] = movies_IMDB_TMDB["revenue"].apply(lambda x: pd.NA if pd.isna(x) or x<=tresh else x)

In [7]:
# Export the combined dataset

cols = ['tconst', 'id', 'primaryTitle', 'originalTitle', 'release_date', 'startYear',
       'runtimeMinutes', 'original_language', 'spoken_languages', 'production_countries', 'genres', 'averageRating', 'numVotes', 
        'budget', 'revenue', 'production_companies', 'overview', 'keywords']
renamed_cols = {
    'tconst': 'imdb_id_movie',
    'id': 'tmdb_id_movie',
    'primaryTitle': 'primaryTitle',
    'originalTitle': 'originalTitle',
    'release_date': 'release_date',
    'startYear': 'release_year',
    'runtimeMinutes': 'runtimeMinutes',
    'original_language': 'original_language',
    'spoken_languages': 'spoken_languages',
    'production_countries': 'production_countries',
    'genres': 'genres',
    'averageRating': 'rating_imdb',
    'numVotes': 'numVotes_imdb',
    'budget': 'budget',
    'revenue': 'revenue',
    'production_companies': 'production_companies',
    'overview': 'overview_tmdb',
    'keywords': 'keywords_tmdb'
}

movies_IMDB_TMDB = movies_IMDB_TMDB[cols].rename(columns=renamed_cols)

movies_IMDB_TMDB.to_csv(DATA_PATH + "movies_imdb_tmdb.tsv", sep='\t', index=False)

# Merge MovieSummaries and Wikidata datasets

In [8]:
# Import and clean MovieSummaries dataset, create key movieTitle_year

cols = ["wikipedia_id_movie", "freebase_id_movie", "title", "release_date", "revenue", "runtimeMinutes",
                       "Movie languages (Freebase ID:name tuples)", "Movie countries (Freebase ID:name tuples)",
                       "Movie genres (Freebase ID:name tuples)"]
movies_original = pd.read_csv(DATA_PATH + 'movie.metadata.tsv', sep='\t', header=None, names=cols).drop(columns="revenue")

movies_original["genres"] = movies_original["Movie genres (Freebase ID:name tuples)"].apply(extract_from_tuple).apply(lowercase)
movies_original["languages"] = movies_original["Movie languages (Freebase ID:name tuples)"].apply(extract_from_tuple).apply(lowercase).apply(remove_language)
movies_original["countries"] = movies_original["Movie countries (Freebase ID:name tuples)"].apply(extract_from_tuple)

movies_original = movies_original.drop(columns=["Movie genres (Freebase ID:name tuples)", "Movie languages (Freebase ID:name tuples)", "Movie countries (Freebase ID:name tuples)"])
movies_original["release_year"] = movies_original["release_date"].apply(extract_year)
movies_original["release_date"] = movies_original["release_date"].apply(lambda x: x if is_valid_date(x) else pd.NA)
movies_original["runtimeMinutes"] = movies_original["runtimeMinutes"].apply(lambda x: pd.NA if pd.isna(x) else int(x))

movies_original["title_year_key"] = create_key_series(movies_original, ["title", "release_year"])
duplicate_keys = movies_original["title_year_key"].value_counts()
duplicate_keys = duplicate_keys[duplicate_keys > 1].index
movies_original["title_year_key"] = movies_original["title_year_key"].apply(lambda x: pd.NA if x in duplicate_keys else x)

In [12]:
# Import and clean Wikidata dataset, create key movieTitle_year

movies_wikidata = pd.read_csv(DATA_PATH + "wikidata_freebaseID_imdbID.csv", dtype={0: str})

movies_wikidata["releaseDate"] = movies_wikidata["releaseDate"].apply(select_releaseDate)
movies_wikidata["year"] = movies_wikidata["releaseDate"].apply(extract_year)
movies_wikidata["year"] = movies_wikidata.apply(lambda row: extract_year(row["description"]) if pd.isna(row["year"]) else row["year"], axis=1)
movies_wikidata["releaseDate"] = movies_wikidata["releaseDate"].apply(lambda x: x if is_valid_date(x) else pd.NA)
movies_wikidata["releaseDate"] = movies_wikidata["releaseDate"].apply(lambda x: x if is_valid_date(x) else pd.NA)
movies_wikidata["languages"] = movies_wikidata["languages"].apply(lowercase)

movies_wikidata["title_year_key"] = create_key_series(movies_wikidata, ["title", "year"])
duplicate_keys = movies_wikidata["title_year_key"].value_counts()
duplicate_keys = duplicate_keys[duplicate_keys > 1].index
movies_wikidata["title_year_key"] = movies_wikidata["title_year_key"].apply(lambda x: pd.NA if x in duplicate_keys else x)

In [13]:
# Merge IMDB and TMDB datasets first by freebaseID and then using the key movieTitle_year

df1 = pd.merge(movies_original, movies_wikidata, left_on="freebase_id_movie", right_on="freebaseID", how="inner", suffixes=('', '_wikidata'))
rest_of_movies_original = movies_original[~movies_original["freebase_id_movie"].isin(df1["freebase_id_movie"])].dropna(subset=["title_year_key"])
df2 = pd.merge(rest_of_movies_original, movies_wikidata, left_on="title_year_key", right_on="title_year_key", how="left", suffixes=('', '_wikidata'))

movies_orig_wiki = pd.concat([df1, df2]).drop(columns=["title_year_key", "title_year_key_wikidata", "title_wikidata"])

In [14]:
# Combine columns that provide the same information

movies_orig_wiki["release_date"] = movies_orig_wiki.apply(lambda row: row["releaseDate"] if pd.isna(row["release_date"]) else row["release_date"], axis=1)
movies_orig_wiki["release_year"] = movies_orig_wiki.apply(lambda row: row["year"] if pd.isna(row["release_year"]) else row["release_year"], axis=1)
movies_orig_wiki["runtimeMinutes"] = movies_orig_wiki.apply(lambda row: row["runtime"] if pd.isna(row["runtimeMinutes"]) else row["runtimeMinutes"], axis=1)
movies_orig_wiki["languages"] = movies_orig_wiki.apply(lambda row: merge_comma_sep(row["languages"], row["languages_wikidata"]), axis=1)
movies_orig_wiki["countries"] = movies_orig_wiki.apply(lambda row: merge_comma_sep(row["countries"], row["countries_wikidata"]), axis=1)

In [15]:
# Export the combined dataset

cols = ['freebase_id_movie', 'wikidataID', 'wikipedia_id_movie', 'imdbID', 'wikipediaLink', 'title', 'description', 'release_date',
        'release_year', 'runtimeMinutes', 'languages', 'countries', 'genres', 'genres_wikidata', 'productionCompanies']

renamed_cols = {
    'freebase_id_movie': 'freebase_id_movie',
    'wikidataID': 'wikidata_id_movie',
    'wikipedia_id_movie': 'wikipedia_id_movie',
    'imdbID': 'imdb_id_movie',
    'wikipediaLink': 'wikipediaLink',
    'title': 'title',
    'runtimeMinutes': 'runtimeMinutes',
    'description': 'description_wikidata',
    'release_date': 'release_date',
    'release_year': 'release_year',
    'runtimeMinutes': 'runtimeMinutes',
    'languages': 'languages',
    'countries': 'countries',
    'genres': 'genres_original',
    'genres_wikidata': 'genres_wikidata',
    'productionCompanies': 'production_companies'
}

movies_orig_wiki = movies_orig_wiki[cols].rename(columns=renamed_cols)

movies_orig_wiki.to_csv(DATA_PATH + "movies_orig_wiki.tsv", sep='\t', index=False)

# Merge IMDB-TMDB and MovieSummaries-Wikidata datasets

In [ ]:
# Import MovieSummaries-Wikidata dataset, create key movieTitle_year

movies_orig_wiki = pd.read_csv(DATA_PATH + "movies_orig_wiki.tsv", sep='\t')

movies_orig_wiki["release_year"] = movies_orig_wiki["release_year"].apply(lambda x: pd.NA if pd.isna(x) else int(x))

movies_orig_wiki["title_year_key"] = create_key_series(movies_orig_wiki, ["title", "release_year"])
duplicate_keys = movies_orig_wiki["title_year_key"].value_counts()
duplicate_keys = duplicate_keys[duplicate_keys > 1].index
movies_orig_wiki["title_year_key"] = movies_orig_wiki["title_year_key"].apply(lambda x: pd.NA if x in duplicate_keys else x)

In [30]:
# Import IMDB-TMDB dataset, create key movieTitle_year

movies_IMDB_TMDB = pd.read_csv(DATA_PATH + "movies_imdb_tmdb.tsv", sep='\t')

movies_IMDB_TMDB["release_year"] = movies_IMDB_TMDB["release_year"].apply(lambda x: pd.NA if pd.isna(x) else int(x))
movies_IMDB_TMDB["spoken_languages"] = movies_IMDB_TMDB["spoken_languages"].apply(lowercase)

movies_IMDB_TMDB["title_year_key"] = create_key_series(movies_IMDB_TMDB, ["primaryTitle", "release_year"])
duplicate_keys = movies_IMDB_TMDB["title_year_key"].value_counts()
duplicate_keys = duplicate_keys[duplicate_keys > 1].index
movies_IMDB_TMDB["title_year_key"] = movies_IMDB_TMDB["title_year_key"].apply(lambda x: pd.NA if x in duplicate_keys else x)

In [31]:
# Merge IMDB-TMDB and MovieSummaries-Wikidata datasets first by imdbID and then using the key movieTitle_year

df1 = pd.merge(movies_orig_wiki, movies_IMDB_TMDB, left_on="imdb_id_movie", right_on="imdb_id_movie", how="inner", suffixes=('', '_IMDB_TMDB'))
rest_of_movies_orig_wiki = movies_orig_wiki[~movies_orig_wiki["imdb_id_movie"].isin(df1["imdb_id_movie"])].dropna(subset=["title_year_key"])
df2 = pd.merge(rest_of_movies_orig_wiki, movies_IMDB_TMDB, left_on="title_year_key", right_on="title_year_key", how="left", suffixes=('', '_IMDB_TMDB'))

movies_complete = pd.concat([df1, df2]).drop(columns=["title_year_key", "title_year_key_IMDB_TMDB"])

In [32]:
# Combine columns that provide the same information

movies_complete["imdb_id_movie"] = movies_complete.apply(lambda row: row["imdb_id_movie_IMDB_TMDB"] if pd.isna(row["imdb_id_movie"]) else row["imdb_id_movie"], axis=1)
movies_complete["primaryTitle"] = movies_complete.apply(lambda row: row["title"] if pd.isna(row["primaryTitle"]) else row["primaryTitle"], axis=1)
movies_complete["release_date"] = movies_complete.apply(lambda row: row["release_date"] if pd.isna(row["release_date_IMDB_TMDB"]) else row["release_date_IMDB_TMDB"], axis=1)
movies_complete["release_year"] = movies_complete.apply(lambda row: row["release_year"] if pd.isna(row["release_year_IMDB_TMDB"]) else row["release_year_IMDB_TMDB"], axis=1)
movies_complete["runtimeMinutes"] = movies_complete.apply(lambda row: row["runtimeMinutes"] if pd.isna(row["runtimeMinutes_IMDB_TMDB"]) else row["runtimeMinutes_IMDB_TMDB"], axis=1)

movies_complete["languages"] = movies_complete.apply(lambda row: merge_comma_sep(row["languages"], row["spoken_languages"]), axis=1)
movies_complete["production_companies"] = movies_complete.apply(lambda row: merge_comma_sep(row["production_companies"], row["production_companies_IMDB_TMDB"]), axis=1)

In [33]:
# Export the combined dataset

cols = ['freebase_id_movie', 'wikidata_id_movie', 'wikipedia_id_movie',
       'imdb_id_movie', 'tmdb_id_movie', 'wikipediaLink', 'primaryTitle', 'originalTitle', 'description_wikidata',
       'release_date', 'release_year', 'runtimeMinutes', 'original_language', 'languages',
       'countries', 'genres_original', 'genres_wikidata', 'genres', 'rating_imdb', 'numVotes_imdb',
       'budget', 'revenue', 'production_companies', 'overview_tmdb', 'keywords_tmdb']

renamed_cols = {
    'genres': 'genres_IMDB_TMDB',
}

movies_complete = movies_complete[cols].rename(columns=renamed_cols).dropna(subset=['release_year'])

movies_complete.to_csv(DATA_PATH + "movies_complete.tsv", sep='\t', index=False)